# Preparing Data Qara'a Qur'an<br>
pertama anda harus datang ke website https://qul.tarteel.ai/resources/recitation lalu mendownload file sqlite.bz2<br>
untuk spesifikasinya yang mana harus <b>Ayah by Ayah<b>, <b>Recitation<b>, <b>Murattal<b>, <b>Hafs<b>, <b>With Segments<b><br>
folder downloadnya letak di folder <i>./bz2/<i>

### 1. Extract Bz2<br>
extract file.bz2 pada folder <i>./bz2/<i> hingga kita mendapatkan file sqlitenya lalu letakkan di folder <i>sqlite/<i>

In [ ]:
import os
import bz2
import shutil

# Folder tempat file berada
folder_path = "./bz2/"  # Ganti dengan lokasi file kamu

# Loop untuk mengekstrak semua file .bz2
for file in os.listdir(folder_path):
    if file.endswith(".bz2"):
        file_path = os.path.join(folder_path, file)
        extracted_path = file_path[:-4]  # Menghapus .bz2 dari nama file

        # Membuka file .bz2 dan menulis hasil ekstraksi
        with bz2.BZ2File(file_path, 'rb') as f_in, open(f"sqlite/{extracted_path}", 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

        print(f"Extracted: {file} -> {extracted_path}")

### 2.Convert to JSON<br>
Ubah file sqlite menjadi JSON serti kita perbaiki urutan angka ayatnya 

In [9]:
import os
import sqlite3
import json

# Define source and destination directories
sqlite_dir = "./sqlite"
json_dir = "./json"

# Create the JSON output directory if it doesn't exist
os.makedirs(json_dir, exist_ok=True)

# Get all SQLite database files
sqlite_files = [f for f in os.listdir(sqlite_dir) if f.endswith(".sqlite")]

def fix_json_structure(input_file, output_file):
    """Memperbaiki struktur JSON agar ayat dalam setiap surah memiliki nomor urut yang benar."""
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    fixed_verses = []
    current_surah = None
    ayah_number = 1

    for verse in data.get("verses", []):
        if verse["surah_number"] != current_surah:
            current_surah = verse["surah_number"]
            ayah_number = 1  # Reset nomor ayat jika surah berubah
        
        verse["ayah_number"] = ayah_number
        fixed_verses.append(verse)
        ayah_number += 1
    
    # Simpan hasil perbaikan ke file baru
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump({"verses": fixed_verses}, f, indent=4, ensure_ascii=False)
    
    print(f"JSON diperbaiki dan disimpan ke: {output_file}")

for sqlite_file in sqlite_files:
    sqlite_path = os.path.join(sqlite_dir, sqlite_file)
    json_path = os.path.join(json_dir, f"{sqlite_file.replace('.sqlite', '.json')}")
    json_fixed_path = os.path.join(json_dir, f"{sqlite_file.replace('.sqlite', '.json')}")

    # Connect to SQLite database
    conn = sqlite3.connect(sqlite_path)
    cursor = conn.cursor()

    # Fetch all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    db_dict = {}

    # Iterate through each table and fetch data
    for table in tables:
        table_name = table[0]
        cursor.execute(f"SELECT * FROM {table_name}")
        columns = [desc[0] for desc in cursor.description]  # Get column names
        rows = cursor.fetchall()
        
        # Convert rows into list of dictionaries
        table_data = [dict(zip(columns, row)) for row in rows]
        
        # Store in dictionary
        db_dict[table_name] = table_data

    # Close connection
    conn.close()

    # Write to JSON file
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(db_dict, json_file, indent=4, ensure_ascii=False)

    print(f"Converted: {sqlite_file} → {json_path}")

    # Fix the JSON structure
    fix_json_structure(json_path, json_fixed_path)

print("Semua SQLite telah dikonversi ke JSON dan diperbaiki!")


Converted: Abdul Basit Abdul Samad.sqlite → ./json/Abdul Basit Abdul Samad.json
JSON diperbaiki dan disimpan ke: ./json/Abdul Basit Abdul Samad.json
Converted: Abdul Rahman Al-Sudais.sqlite → ./json/Abdul Rahman Al-Sudais.json
JSON diperbaiki dan disimpan ke: ./json/Abdul Rahman Al-Sudais.json
Converted: Abu Bakr al-Shatri.sqlite → ./json/Abu Bakr al-Shatri.json
JSON diperbaiki dan disimpan ke: ./json/Abu Bakr al-Shatri.json
Converted: Hani ar-Rifai Recitation.sqlite → ./json/Hani ar-Rifai Recitation.json
JSON diperbaiki dan disimpan ke: ./json/Hani ar-Rifai Recitation.json
Converted: Khalifa Al Tunaiji.sqlite → ./json/Khalifa Al Tunaiji.json
JSON diperbaiki dan disimpan ke: ./json/Khalifa Al Tunaiji.json
Converted: Maher Al-Mu'aiqly.sqlite → ./json/Maher Al-Mu'aiqly.json
JSON diperbaiki dan disimpan ke: ./json/Maher Al-Mu'aiqly.json
Converted: Mahmoud Khalil Al-Husary.sqlite → ./json/Mahmoud Khalil Al-Husary.json
JSON diperbaiki dan disimpan ke: ./json/Mahmoud Khalil Al-Husary.json
Co